# Creating a Database of Company Filings from EDGAR

In [34]:
import datetime
import os
import pandas as pd
import random
import re
import requests
import string
import time

from bs4 import BeautifulSoup
from io import StringIO

COMPANY_REGEX = re.compile(r'^company\.idx$')
QUARTER_REGEX = re.compile(r'^QTR[1-4]$')
YEAR_REGEX = re.compile(r'^\d{4}$')
EDGAR_BASE_URL = "https://www.sec.gov/Archives"  # all .idx urls are relative to this
EDGAR_INDEX_URL = "https://www.sec.gov/Archives/edgar/full-index/"

DOWNLOAD_PATH = "/home/mallinger/Coding/notebooks/data/edgar"

## Scrape the Index Locations

The EDGAR indexes are behind two layers of web pages.  The first (https://www.sec.gov/Archives/edgar/full-index/) provides a table of contents for each year.  The second (1997 example: https://www.sec.gov/Archives/edgar/full-index//1997/) provides a table of contents for the quarters in each year.

Inside the quarter directories are many files.  But the .idx files are all the same, simply ordered by different criteria.  So we'll take "company.idx" to get the index for each quarter.

In [ ]:
def parse_index_page(url, link_re, base_url=None):
    """Given a URL to a contents page, returns all the links that match
    the provided regular expression.  Helpful for scraping table of
    contents pages.
    """
    index = requests.get(url)
    soup = BeautifulSoup(index.content, 'html.parser')
    # all content links on the EDGAR pages are inside the main area
    # designated by this HTML id attribute.
    content_links = soup.find(id="main-content").find_all('a')
    urls = []
    for link in content_links:
        if link_re.match(link.getText()):
            href = link.get("href")
            if base_url:
                href = "{}/{}".format(base_url, href)
            urls.append(href)
    return urls

def parse_all_index_pages(urls, link_re):
    """Given a list of URLs to a contents pages, returns a list of
    all the links found that match the link regular expression.
    """
    found_urls = []
    for url in urls:
        found_urls.extend(parse_index_page(url, link_re, base_url=url))
    return found_urls

# get all the URLs to year partitions
year_urls = parse_index_page(EDGAR_INDEX_URL, YEAR_REGEX, base_url=EDGAR_INDEX_URL)
# get all the URLs to the quarter partitions
quarter_urls = parse_all_index_pages(year_urls, QUARTER_REGEX)
# get the company.idx file in the quarterly folder
index_urls = parse_all_index_pages(quarter_urls, COMPANY_REGEX)

## Create the Index

Note that this code is extremely memory intensive as the number of records is in the 10's of millions.  If you don't have over 10gb of memory free, you may want to edit it into batches.

In [ ]:
def parse_index_file(url):
    """Given a URL to an .idx Edgar file, reads the index and returns
    a Pandas dataframe of the results.
    """
    # note: work with bytes strings here as the offsets for fixed width columns
    # get thrown off if there are conversion errors or bad characters (There are!)
    rows = requests.get(url).content.split(b"\n")
    # there is a header consisting of leading rows with comments followed by
    # a row of hyphens to start the data (------)
    row_offset = 1 + min([i for i, row in enumerate(rows) if re.match(b"------", row)])
    rows = rows[row_offset:]
    # skip any empty rows
    rows = list(filter(lambda row: not re.match(b"^\s*$", row), rows))
        
    # note, we can't use pd.read_fwf because of invalid characters in the bytes records.
    # this approach avoids lost records
    array_frame = [(row[0:62].strip().decode("utf-8", "ignore"),
                    row[62:74].strip().decode("utf-8", "ignore"),
                    row[74:86].strip().decode("utf-8", "ignore"),
                    row[86:98].strip().decode("utf-8", "ignore"),
                    row[98:].strip().decode("utf-8", "ignore")) 
                   for row in rows]
    df = pd.DataFrame(array_frame, columns=["name", "form", "cik", "date", "url"])
    df["date"] = pd.to_datetime(df["date"])
    return df

index = None
for url in index_urls:
    df = parse_index_file(url)
    if index is None:
        index = df
    else:
        index = pd.concat([index, df], ignore_index=True)


## Save the Index

In [ ]:
index.to_pickle(os.path.join(DOWNLOAD_PATH, "edgar_index.pickle.gzip"), compression="gzip")

## Load the Index

(This cell for ease of starting with saved data)

In [2]:
index = pd.read_pickle(os.path.join(DOWNLOAD_PATH, "edgar_index.pickle.gzip"), compression="gzip")

In [ ]:
is_form_424b2 = index["form"].str.contains("424B2")

# create a dataset to download
records = index[is_form_424b2]
counter = 0
for _, record in records.iterrows():
    url = "{}/{}".format(EDGAR_BASE_URL, record["url"])
    content = requests.get(url).content.decode("utf-8", "ignore")
    # add a random padding to avoid filename colisions
    padding = ''.join([random.choice(string.ascii_letters + string.digits) for i in range(5)])
    filename = "424b-{cik}-{date}-{padding}.txt".format(
                    cik=record["cik"],
                    date=record["date"].strftime("%Y%m%d"),
                    padding=padding)
    
    with open(os.path.join(DOWNLOAD_PATH, filename), "w") as fh:
        fh.write(content)
        
    # sleep for a second every 10 records
    counter += 1
    if counter % 10 == 0:
        time.sleep(1)

In [23]:

# content = b"\n".join(rows[row_offset:])
# return pd.read_fwf(StringIO(content), 
#                    colspecs=[(0,62), (62,74), (74,86), (86,98), (98, 151)], 
#                    names=["company", "form", "cik", "date", "url"])